In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#環境確認
import pandas as pd
import numpy as np
import sklearn
import gensim
import spacy
import re
!python3 --version
print(pd.__version__)
print(np.__version__)
print(sklearn.__version__)
import matplotlib
print(matplotlib.__version__)
print(gensim.__version__)

import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
import seaborn as sns
import string

Python 3.7.10
1.1.5
1.19.5
0.22.2.post1
3.2.2
3.6.0


In [4]:
from nltk.corpus import stopwords
import nltk
from bs4 import BeautifulSoup
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
# StopWord の再定義
stop = set(stopwords.words("english"))

# 句読点の追加。string.punctuation = ['!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~']
punctuation = list(string.punctuation)

# 手動で追加
org_stop = ["Subject"]

# stopwordsの定義更新
add_stop = punctuation# + org_stop
stop.update(add_stop)

In [62]:
# htmlの分割
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

# []で囲まれた文章の削除（脚注、linkなど）
def remove_between_square_brackets(text):
    return re.sub('\[[^[]*\]', '', text)

def remove_between_round_brackets(text):
    return re.sub('\([^()]*\)', '', text)

# URLの削除
def remove_URLs(text):
    return re.sub(r'http\S+', '', text)

def replace_eq(text):
    return re.sub('\$[^$]+\=[^$]+\$', 'the equation ', text)

def replace_symbol(text):
    return re.sub('\$[^$]*\w\$', 'the symbol ', text)

def replace_symbol_plus(text):
    return re.sub('\{[^$]*\w\}', 'the symbol ', text)

def remove_newline(text):
    return re.sub('\\n', ' ', text)


# stopwordsの削除
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stop:
            if i.strip().isalpha():
                final_text.append(i.strip())
    return " ".join(final_text)

def remove_puncts(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in punctuation:
            if i.strip().isalpha():
                final_text.append(i.strip())
    return " ".join(final_text).lower()

# 上記の関数をまとめて適用する関数を定義
def denoise_text(text):
    text = strip_html(text)
    text = remove_URLs(text)
    text = remove_between_square_brackets(text)
    text = remove_between_round_brackets(text)
    text = replace_eq(text)
    text = replace_symbol(text)
    text = replace_symbol_plus(text)
    text = remove_newline(text)
    #text = remove_stopwords(text)
    text = remove_puncts(text)
    return text

In [63]:
#CSVをデータフレームに取り込む
df = pd.read_csv('/content/drive/MyDrive/abstract/arxiv_abstract.csv', header=None, names=['date', 'abst'])
#'/content/drive/MyDrive/kaggle/sampled_abst_stacked4.txt'

In [64]:
print(df.head(10))

                        date                                               abst
0  2021-05-06 16:39:15+00:00  Fermi liquid theory forms the basis for our un...
1  2021-05-06 16:29:18+00:00  We analyze Andreev bound states (ABSs) that fo...
2  2021-05-06 14:38:49+00:00  We have investigated the charge ordering phase...
3  2021-05-06 14:02:51+00:00  We derive a series of quantitative bulk-bounda...
4  2021-05-06 13:58:30+00:00  The van der Waals magnets provide an ideal pla...
5  2021-05-06 13:20:17+00:00  Impurity pinning has long been discussed to ha...
6  2021-05-06 10:19:24+00:00  We investigate the use of programmable optical...
7  2021-05-06 05:40:43+00:00  Dynamical quantum phase transitions (DQPTs), w...
8  2021-05-06 03:18:21+00:00  Recent experiments have observed correlated in...
9  2021-05-06 03:08:20+00:00  Recently, quantum oscillation of the resistanc...


In [65]:
#abstに文字処理の関数を適用
df['abst'] = df['abst'].apply(denoise_text)

In [66]:
for text in df['abst'][97:99]:
    print(text)

we report on depinning of waves in at room a combination of the differential measurements with the noise spectroscopy provide unambiguous means for detecting the depinning threshold field in the depinning process in is not accompanied by an observable abrupt increase in electric current in striking contrast to depinning in the conventional materials with crystal we explained it by the fact that the current density from the waves in the devices is orders of magnitude smaller than the current density of the free carriers available in the discommensuration network surrounding the commensurate the depinning fields in devices are several orders of magnitude larger than those in van der waals obtained results are important for the proposed applications of the devices in
polarons with different types of coupling have fundamentally different when the dominant interaction is between the electron density and lattice the momentum of the ground state does not change and the polaron gets exponentia

In [67]:
print(len(df))

20000


In [74]:
#データフレームの各行のAbstを一文に統合。各Abst間はスペースでつなぐ。
stacked_text = ''
for sent in df['abst']:
    stacked_text += sent
    #stacked_text += ' </s>' 
    stacked_text += ' ' 

In [75]:
len(stacked_text.split(' '))

2562516

In [76]:
with open('/content/drive/MyDrive/abstract/condmat_abst.txt', mode='w') as f:
    f.write(stacked_text)

In [21]:
!git clone https://github.com/facebookresearch/fastText.git

Cloning into 'fastText'...
remote: Enumerating objects: 3854, done.
remote: Total 3854 (delta 0), reused 0 (delta 0), pack-reused 3854
Receiving objects: 100% (3854/3854), 8.22 MiB | 33.69 MiB/s, done.
Resolving deltas: 100% (2417/2417), done.


In [22]:
%cd fastText

/content/fastText


In [23]:
!pwd

/content/fastText


In [24]:
!pip install .

Processing /content/fastText
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3088895 sha256=293850bf06bc682d9fe67695498a11c768784bad51f77757ec284ee60e69d79a
  Stored in directory: /tmp/pip-ephem-wheel-cache-nooxeriz/wheels/a1/9f/52/696ce6c5c46325e840c76614ee5051458c0df10306987e7443
Successfully built fasttext


In [25]:
%cd /content

/content


In [26]:
#https://fasttext.cc/docs/en/python-module.html#model-object
import fasttext

In [77]:
model = fasttext.train_unsupervised('/content/drive/MyDrive/abstract/condmat_abst.txt') #学習実行。学習量次第で時間かかる。
#model = fasttext.load_model('/content/drive/MyDrive/abstract/model_all_abst.bin')#一度学習したあとはこちらのBinを利用すると早い。

In [108]:
model.get_nearest_neighbors("cuprates") #入力単語に類似した単語の表示

[(0.9023925065994263, 'cuprate'),
 (0.7305797934532166, 'underdoped'),
 (0.708753764629364, 'underdoping'),
 (0.6683498024940491, 'overdoped'),
 (0.6558646559715271, 'pseudogaps'),
 (0.653243362903595, 'nickelates'),
 (0.6503386497497559, 'pseudogap'),
 (0.632213294506073, 'dichotomy'),
 (0.6306652426719666, 'cdm'),
 (0.6301589608192444, 'mystery')]

In [100]:
A="physics"
B="superconductivity"
C="life"
model.get_analogies(A,B,C,k=10) #A-B+C：単語ベクトルの演算、ｋは表示数

[(0.5835455656051636, 'astrophysics'),
 (0.5238144993782043, 'lifep'),
 (0.4930766224861145, 'matters'),
 (0.4795878529548645, 'topics'),
 (0.4706553816795349, 'basics'),
 (0.46618160605430603, 'problems'),
 (0.45031604170799255, 'forefront'),
 (0.44582653045654297, 'aspects'),
 (0.44469180703163147, 'lifeas'),
 (0.44305211305618286, 'paradigms')]

In [30]:
model.save_model('/content/drive/MyDrive/abstract/model_all_abst.bin')